In [ ]:
import pandas as pd
import numpy as np

In [ ]:
print(ball_df.columns)

In [ ]:
# Read the CSV file
ball_df = pd.read_csv('../data/raw/ball_positions.csv')

# Strip leading/trailing spaces from column names
ball_df.columns = ball_df.columns.str.strip()

# Debugging: Print the cleaned column names
print("Cleaned Columns:", ball_df.columns)

# Interpolate missing X and Y values
ball_df[['X', 'Y']] = ball_df[['X', 'Y']].interpolate(limit_direction='both')

# Drop rows where X or Y are still NaN
ball_df = ball_df.dropna(subset=['X', 'Y'])

# Convert X and Y to float32
ball_df[['X', 'Y']] = ball_df[['X', 'Y']].astype('float32')

# Check result
print("After interpolation:")
print(ball_df.head(10))

# Save cleaned version (optional)
ball_df.to_csv("../data/results/ball_positions_interpolated.csv", index=False)